In [1]:
import time
import random
import numpy as np
import pickle as pkl
import tensorflow as tf

from keras.models import Model
from tensorflow.keras.layers import Input
from keras.layers import Embedding, Dense, Dropout, LSTM, CuDNNLSTM, Bidirectional, TimeDistributed
from keras.optimizers import Adam
from keras.utils import Sequence
from keras.initializers import glorot_normal
from keras.callbacks import ModelCheckpoint

In [2]:
WITH_EXTRA_TRAIN = False


with open("ARABIC_LETTERS_LIST.pickle", 'rb') as file:
    ARABIC_LETTERS_LIST = pkl.load(file)
with open('DIACRITICS_LIST.pickle', 'rb') as file:
    DIACRITICS_LIST = pkl.load(file)
if not WITH_EXTRA_TRAIN:
    with open('RNN_SMALL_CHARACTERS_MAPPING.pickle', 'rb') as file:
        CHARACTERS_MAPPING = pkl.load(file)
else:
    with open('RNN_BIG_CHARACTERS_MAPPING.pickle', 'rb') as file:
        CHARACTERS_MAPPING = pkl.load(file)
with open('RNN_CLASSES_MAPPING.pickle', 'rb') as file:
    CLASSES_MAPPING = pkl.load(file)
with open('RNN_REV_CLASSES_MAPPING.pickle', 'rb') as file:
    REV_CLASSES_MAPPING = pkl.load(file)

In [3]:
train_raw = None
with open('train.txt','r') as file:
    train_raw = file.readlines()
if WITH_EXTRA_TRAIN:
    with open('extra_train.txt', 'r') as file:
        train_raw += file.readlines()
print('Training examples (raw):', len(train_raw))

val_raw = None
with open('val.txt', 'r') as file:
    val_raw = file.readlines()
print('Validation examples (raw):', len(val_raw))

Training examples (raw): 50000
Validation examples (raw): 2500


In [4]:
def remove_diacritics(data_raw):
    return data_raw.translate(str.maketrans('', '', ''.join(DIACRITICS_LIST)))

In [5]:
def to_one_hot(data, size):
    one_hot = list()
    for elem in data:
        cur = [0] * size
        cur[elem] = 1
        one_hot.append(cur)
    return one_hot

In [6]:
def split_data(data_raw):
    data_new = list()
    
    for line in data_raw:
        line = line.replace('.', '.\n')
        line = line.replace(',', ',\n')
        line = line.replace('،', '،\n')
        line = line.replace(':', ':\n')
        line = line.replace(';', ';\n')
        line = line.replace('؛', '؛\n')
        line = line.replace('(', '\n(')
        line = line.replace(')', ')\n')
        line = line.replace('[', '\n[')
        line = line.replace(']', ']\n')
        line = line.replace('{', '\n{')
        line = line.replace('}', '}\n')
        line = line.replace('«', '\n«')
        line = line.replace('»', '»\n')
        
        for sub_line in line.split('\n'):
            if len(remove_diacritics(sub_line).strip()) == 0:
                continue
            
            if len(remove_diacritics(sub_line).strip()) > 0 and len(remove_diacritics(sub_line).strip()) <= 500:
                data_new.append(sub_line.strip())
            else:
                sub_line = sub_line.split()
                tmp_line = ''
                for word in sub_line:
                    if len(remove_diacritics(tmp_line).strip()) + len(remove_diacritics(word).strip()) + 1 > 500:
                        if len(remove_diacritics(tmp_line).strip()) > 0:
                            data_new.append(tmp_line.strip())
                        tmp_line = word
                    else:
                        if tmp_line == '':
                            tmp_line = word
                        else:
                            tmp_line += ' '
                            tmp_line += word
                if len(remove_diacritics(tmp_line).strip()) > 0:
                    data_new.append(tmp_line.strip())

    return data_new

In [7]:
train_split = split_data(train_raw)
val_split = split_data(val_raw)

In [8]:
print('Training examples (split):', len(train_split))
print('Validation examples (split):', len(val_split))

Training examples (split): 299645
Validation examples (split): 14698


In [9]:
def map_data(data_raw):
    X = list()
    Y = list()
    
    for line in data_raw:        
        x = [CHARACTERS_MAPPING[' ']]
        y = [CLASSES_MAPPING['']]
        
        for idx, char in enumerate(line):
            if char in DIACRITICS_LIST:
                continue
            
            x.append(CHARACTERS_MAPPING[char])
            
            if char not in ARABIC_LETTERS_LIST:
                y.append(CLASSES_MAPPING[''])
            else:
                char_diac = ''
                if idx + 1 < len(line) and line[idx + 1] in DIACRITICS_LIST:
                    char_diac = line[idx + 1]
                    if idx + 2 < len(line) and line[idx + 2] in DIACRITICS_LIST and char_diac + line[idx + 2] in CLASSES_MAPPING:
                        char_diac += line[idx + 2]
                    elif idx + 2 < len(line) and line[idx + 2] in DIACRITICS_LIST and line[idx + 2] + char_diac in CLASSES_MAPPING:
                        char_diac = line[idx + 2] + char_diac
                y.append(CLASSES_MAPPING[char_diac])
        
        assert(len(x) == len(y))
        
        x.append(CHARACTERS_MAPPING[' '])
        y.append(CLASSES_MAPPING[''])
        
        y = to_one_hot(y, len(CLASSES_MAPPING))
        
        X.append(x)
        Y.append(y)
    
    X = np.asarray(X)
    Y = np.asarray(Y)
    
    return X, Y

In [10]:
def create_model():
    SelectedLSTM = LSTM
  
    inputs = Input(shape=(None,))
    
    embeddings = Embedding(input_dim=len(CHARACTERS_MAPPING),
                           output_dim=25,
                           embeddings_initializer=glorot_normal(seed=961))(inputs)
    
    blstm1 = Bidirectional(SelectedLSTM(units=256,
                                     return_sequences=True,
                                     kernel_initializer=glorot_normal(seed=961)))(embeddings)
    dropout1 = Dropout(0.5)(blstm1)
    blstm2 = Bidirectional(SelectedLSTM(units=256,
                                     return_sequences=True,
                                     kernel_initializer=glorot_normal(seed=961)))(dropout1)
    dropout2 = Dropout(0.5)(blstm2)
    blstm3 = Bidirectional(SelectedLSTM(units=256,
                                     return_sequences=True,
                                     kernel_initializer=glorot_normal(seed=961)))(dropout2)
    dropout3 = Dropout(0.5)(blstm3)
    
    dense1 = TimeDistributed(Dense(units=512,
                                   activation='relu',
                                   kernel_initializer=glorot_normal(seed=961)))(dropout2)
    dense2 = TimeDistributed(Dense(units=512,
                                   activation='relu',
                                   kernel_initializer=glorot_normal(seed=961)))(dense1)
    
    output = TimeDistributed(Dense(units=len(CLASSES_MAPPING),
                                   activation='softmax',
                                   kernel_initializer=glorot_normal(seed=961)))(dense2)
    
    model = Model(inputs, output)
    
    model.compile(loss='categorical_crossentropy', optimizer=Adam())
    
    return model

In [11]:
model = create_model()
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding (Embedding)       (None, None, 25)          1925      
                                                                 
 bidirectional (Bidirectiona  (None, None, 512)        577536    
 l)                                                              
                                                                 
 dropout (Dropout)           (None, None, 512)         0         
                                                                 
 bidirectional_1 (Bidirectio  (None, None, 512)        1574912   
 nal)                                                            
                                                                 
 dropout_1 (Dropout)         (None, None, 512)         0     

In [12]:
class DataGenerator(Sequence):
    def __init__(self, lines, batch_size):
        self.lines = lines
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.lines) / float(self.batch_size)))

    def __getitem__(self, idx):
        lines = self.lines[idx * self.batch_size:(idx + 1) * self.batch_size]
        X_batch, Y_batch = map_data(lines)
        
        X_max_seq_len = np.max([len(x) for x in X_batch])
        Y_max_seq_len = np.max([len(y) for y in Y_batch])
        
        assert(X_max_seq_len == Y_max_seq_len)
        
        X = list()
        for x in X_batch:
            x = list(x)
            x.extend([CHARACTERS_MAPPING[' ']] * (X_max_seq_len - len(x)))
            X.append(np.asarray(x))
        
        Y_tmp = list()
        for y in Y_batch:
            y_new = list(y)
            y_new.extend(to_one_hot([CLASSES_MAPPING['']] * (Y_max_seq_len - len(y)), len(CLASSES_MAPPING)))
            Y_tmp.append(np.asarray(y_new))
        Y_batch = Y_tmp
        
        Y_batch = np.asarray(Y_batch)
        
        return np.asarray(X), Y_batch

In [13]:
def fit_model(model, epochs, batch_size, train_split, val_split):
    random.shuffle(train_split)
    train_split = list(sorted(train_split, key=lambda line: len(remove_diacritics(line))))
    random.shuffle(val_split)
    val_split = list(sorted(val_split, key=lambda line: len(remove_diacritics(line))))
        
    checkpoint_path = 'checkpoints/epoch{epoch:02d}.ckpt'
    checkpoint_cb = ModelCheckpoint(checkpoint_path, verbose=0)
    
    training_generator = DataGenerator(train_split, batch_size)
    val_generator = DataGenerator(val_split, batch_size)

    model.fit_generator(generator=training_generator,
                        validation_data=val_generator,
                        epochs=epochs,
                        callbacks=[checkpoint_cb])

In [14]:
start_time = time.time()
fit_model(model, 50, 256, train_split, val_split)
end_time = time.time()
print('--- %s seconds ---' % round(end_time - start_time, 2))

<ipython-input-13-2a6cf52736ac>:13: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generator=training_generator,


Epoch 1/50
   2/1171 [..............................] - ETA: 5:56 - loss: 2.9205   

<ipython-input-9-55de7b534440>:37: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X = np.asarray(X)
<ipython-input-9-55de7b534440>:38: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  Y = np.asarray(Y)


1171/1171 [==============================] - ETA: 0s - loss: 0.5745

1171/1171 [==============================] - 137s 108ms/step - loss: 0.5745 - val_loss: 0.2378
Epoch 2/50
   2/1171 [..............................] - ETA: 7:20 - loss: 0.1762

...

1171/1171 [==============================] - ETA: 0s - loss: 0.0409

1171/1171 [==============================] - 136s 117ms/step - loss: 0.0409 - val_loss: 0.0673
Epoch 50/50


<ipython-input-9-55de7b534440>:37: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X = np.asarray(X)
<ipython-input-9-55de7b534440>:38: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  Y = np.asarray(Y)


1171/1171 [==============================] - ETA: 0s - loss: 0.0404

1171/1171 [==============================] - 136s 116ms/step - loss: 0.0404 - val_loss: 0.0691
--- 6800.14 seconds ---


# New section

# New section

In [15]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [16]:
def predict(line, model):
    X, _ = map_data([line])
    predictions = model.predict(X).squeeze()
    predictions = predictions[1:]
    
    output = ''
    for char, prediction in zip(remove_diacritics(line), predictions):
        output += char
        
        if char not in ARABIC_LETTERS_LIST:
            continue
        
        if '<' in REV_CLASSES_MAPPING[np.argmax(prediction)]:
            continue

        output += REV_CLASSES_MAPPING[np.argmax(prediction)]

    return output

In [17]:
print(predict('اللهم علمنا ما ينفعنا وإنفعنا بما علمتنا إنك أنت العليم الحكيم', model))

1/1 [==============================] - 3s 3s/step
اللَّهُمَّ عَلِّمْنَا مَا يَنْفَعُنَا وَإِنْفَعْنَا بِمَا عَلِمْتنَا إِنَّك أَنْتِ الْعَلِيمُ الْحَكِيمُ


In [18]:
with open('test.txt', 'r') as file:
    lines = file.readlines()

In [19]:
results = list()
for idx, line in enumerate(lines):
    line = remove_diacritics(line.strip())
    x = predict(line, model)
    results.append(x)

1/1 [==============================] - 0s 34ms/step


In [20]:
with open('outputs.txt', 'w') as file:
    file.write('\n'.join(results))

save the model

In [21]:

import pickle
filename = 'rnnmodel.sav'
pickle.dump(model, open(filename, 'wb'))

Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers
......bidirectional
.........backward_layer
............cell
...............vars
..................0
..................1
..................2
............vars
.........forward_layer
............cell
...............vars
..................0
..................1
..................2
............vars
.........layer
............cell
...............vars
............vars
.........vars
......bidirectional_1
.........backward_layer
............cell
...............vars
..................0
..................1
..................2
............vars
.........forward_layer
............cell
...............vars
..................0
..................1
..................2
............vars
.........layer
............cell
...............vars
............vars
.........vars
......dropout
.........vars
......dropout_1
.........vars
......embedding
.........vars
............0
......input_layer
.........vars
......time_distributed
.........

In [22]:
!pip install git+https://www.github.com/keras-team/keras-contrib.git


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-xwj2ki2z
  Running command git clone --filter=blob:none --quiet https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-xwj2ki2z
  Resolved https://www.github.com/keras-team/keras-contrib.git to commit 3fc5ef709e061416f4bc8a92ca3750c824b5d2b0
  Preparing metadata (setup.py) ... done
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-py3-none-any.whl size=101076 sha256=9ba4662897b945ea1070295c5d484c92e383e8f784a4e05771c4e46c528d9845
  Stored in directory: /tmp/pip-ephem-wheel-cache-w7ykauaz/wheels/67/d2/f4/96ae3c3c62d1e05abfc8860ad0c1207794726d44ebbbb547f3
Successfully built keras-contrib


In [23]:
!pip install keras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [24]:
pip install onnx==1.8

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 10.0 MB/s eta 0:00:00


In [25]:
!pip install PyStemmer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.0/303.0 KB 17.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for PyStemmer: filename=PyStemmer-2.2.0.1-cp38-cp38-linux_x86_64.whl size=599196 sha256=c62cf7bfbae707c36c16d0ea3a47af0f4a67ca99ba0513bc01b2756353eafe9b
  Stored in directory: /root/.cache/pip/wheels/78/04/32/a81f10f01775fcadba622dbcf8305f8053ab1db21b20a25fc4
Successfully built PyStemmer


In [ ]:
!pip install pyarabic
!pip install emoji==1.7.0
!pip install pystemmer
!pip install optuna==2.3.0
!pip install transformers==4.2.1

In [ ]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))
    !nvidia-smi

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

In [28]:
import numpy as np
import pandas as pd
import pyarabic.araby as ar

import re , emoji, Stemmer, functools, operator, string
import torch , optuna, gc, random, os

from tqdm import tqdm_notebook as tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, f1_score, confusion_matrix, precision_score , recall_score
from transformers import AutoConfig, AutoModelForSequenceClassification, AutoTokenizer
from transformers.data.processors import SingleSentenceClassificationProcessor
from transformers import Trainer , TrainingArguments
from transformers.trainer_utils import EvaluationStrategy
from transformers.data.processors.utils import InputFeatures
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from sklearn.utils import resample

import logging

logging.basicConfig(level=logging.WARNING)
logger = logging.getLogger(__name__)

In [29]:
st =  Stemmer.Stemmer('arabic')
def data_cleaning (text):
  text = re.sub(r'^https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
  text = re.sub(r'^http?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
  text = re.sub(r"http\S+", "", text)
  text = re.sub(r"https\S+", "", text)
  text = re.sub(r'\s+', ' ', text)
  text = re.sub("(\s\d+)","",text) 
  text = re.sub(r"$\d+\W+|\b\d+\b|\W+\d+$", "", text)
  text = re.sub("\d+", " ", text)
  text = ar.strip_tashkeel(text)
  text = ar.strip_tatweel(text)
  text = text.replace("#", " ");
  text = text.replace("@", " ");
  text = text.replace("_", " ");
  translator = str.maketrans('', '', string.punctuation)
  text = text.translate(translator)
  em = text
  em_split_emoji = emoji.get_emoji_regexp().split(em)
  em_split_whitespace = [substr.split() for substr in em_split_emoji]
  em_split = functools.reduce(operator.concat, em_split_whitespace)
  text = " ".join(em_split)
  text = re.sub(r'(.)\1+', r'\1', text)
  text_stem = " ".join([st.stemWord(i) for i in text.split()])
  text = text +" "+ text_stem
  text = text.replace("آ", "ا")
  text = text.replace("إ", "ا")
  text = text.replace("أ", "ا")
  text = text.replace("ؤ", "و")
  text = text.replace("ئ", "ي")
   
  return text

In [30]:
# Define the variables needed:

Tweets_Ids_Col_Train ="Tweet_id"
Tweets_Text_Col_Train = "Text"
Tweets_Sentiment_Col_Train = "sentiment"
Train_Data_Extended_File = "train_all_ext.csv"

train_data = pd.DataFrame()
train_data = pd.read_csv(Train_Data_Extended_File, sep=",")


print(train_data[Tweets_Sentiment_Col_Train].value_counts())
print(train_data.value_counts())

neutral     50527
negative    11637
positive    11450
Name: sentiment, dtype: int64
Tweet_id             sentiment  Text                                                                                                                              
197552168667054080   neutral    #صباح_العربية عمره تجاوز المئة وخطف بطولة العالم في سباق الدراجات الهوائيه لكبار السن يبدأ يومه بنصف ساعة رياضة وتزوج أربع مرات       1
1226407928397205505  neutral    #صباح_الخير يارب أيام حلوة تورّد اللي بداخلنَا مثل دُعاء يستجَاب أو أمُنية تتحقّق ..                                                  1
1226408104478244865  neutral    لو حطيت نفسك في كل موقف سيئ في مكان اللي قدامكك ستجد الف مبرر لهذا التصرف ... #صباح_الخير                                             1
1226408035045793793  neutral    ذات صباح شتوي كنت انتظره يدثرني بكلماته دفئاً ... #صباح_الخير                                                                         1
1226408030738243584  neutral    .. آلآبتسآمة .. علوان نقاء النفس وصفائها#صباح_الخ

In [31]:
# Cleaning Training Data 
train_data[Tweets_Text_Col_Train] = train_data[Tweets_Text_Col_Train].apply(lambda x:   data_cleaning(x))

# Removing un-needed feilds
if Tweets_Ids_Col_Train in train_data.columns:
  del train_data[Tweets_Ids_Col_Train]
train_data.columns = [Tweets_Sentiment_Col_Train,Tweets_Text_Col_Train]

train_data[Tweets_Text_Col_Train].head(50)

<ipython-input-29-f0cb5da77b0e>:19: DeprecationWarning: 'emoji.get_emoji_regexp()' is deprecated and will be removed in version 2.0.0. If you want to remove emoji from a string, consider the method emoji.replace_emoji(str, replace='').
To hide this warning, pin/downgrade the package to 'emoji~=1.6.3'
  em_split_emoji = emoji.get_emoji_regexp().split(em)


0     الزعل بيغير ملامحك بيغير نظرة العين بيغير شكلك...
1     halgawi DmfMohe ليس حبا في ايران بقدر ماهو نكا...
2     adalfahaduwail ابي اعرف الحاكم العربي المسلم ا...
3     sarmadbouchamou DimaSadek في الخطاب تبع سليم س...
4     FofaMahmoud مفيش الكلام ده في الزمن FofaMahmou...
5     elm عندي مشكله لما تبي اجد اقامة عامل حاولت ات...
6     الرياسة الفلسطينية تدعو السفراء العرب والمسلمي...
7     Hamed Alali التجنيس يضر اهل السنه في سوريا تغي...
8        سوف تبحث عني في شخص اخر سوف تبحث عن في شخص اخر
9     Bethoven كل ما املكه في تلك الحياة هو كرامتي و...
10    ان كان لك نصيب في شيء سيقلب اله كل الموازين لك...
11    الموكد ان هذا الوزيز هو معمر العرياني والموكد ...
12    الاتحادالافريقي لكرة القدم caf fifa دى جماهير ...
13    mounirhafi في واحد كان معبى الكراسي وزراء وطلع...
14    لاغاب نور القمر في عتمة اليل ينشاف نور البدر ف...
15    عطيتك قلب م عطيته لاحد غيرك غلطه و في ذمتي م ك...
16    الغريب في الامر انك مازلت تنتظر رغم كل هذه الخ...
17    almobark عقبال القمة دنيا باطما تاجر فبات 

In [32]:
train_data[Tweets_Text_Col_Train].shape

(73614,)

In [33]:
train_data1 = pd.read_csv(Train_Data_Extended_File, sep=",")
# Cleaning Training Data 
train_data1[Tweets_Text_Col_Train] = train_data1[Tweets_Text_Col_Train].apply(lambda x:   data_cleaning(x))

# Removing un-needed feilds
if Tweets_Ids_Col_Train in train_data1.columns:
  del train_data1[Tweets_Ids_Col_Train]
train_data1.columns = [Tweets_Sentiment_Col_Train,Tweets_Text_Col_Train]

train_data1[Tweets_Text_Col_Train].head(50)

<ipython-input-29-f0cb5da77b0e>:19: DeprecationWarning: 'emoji.get_emoji_regexp()' is deprecated and will be removed in version 2.0.0. If you want to remove emoji from a string, consider the method emoji.replace_emoji(str, replace='').
To hide this warning, pin/downgrade the package to 'emoji~=1.6.3'
  em_split_emoji = emoji.get_emoji_regexp().split(em)


0     الزعل بيغير ملامحك بيغير نظرة العين بيغير شكلك...
1     halgawi DmfMohe ليس حبا في ايران بقدر ماهو نكا...
2     adalfahaduwail ابي اعرف الحاكم العربي المسلم ا...
3     sarmadbouchamou DimaSadek في الخطاب تبع سليم س...
4     FofaMahmoud مفيش الكلام ده في الزمن FofaMahmou...
5     elm عندي مشكله لما تبي اجد اقامة عامل حاولت ات...
6     الرياسة الفلسطينية تدعو السفراء العرب والمسلمي...
7     Hamed Alali التجنيس يضر اهل السنه في سوريا تغي...
8        سوف تبحث عني في شخص اخر سوف تبحث عن في شخص اخر
9     Bethoven كل ما املكه في تلك الحياة هو كرامتي و...
10    ان كان لك نصيب في شيء سيقلب اله كل الموازين لك...
11    الموكد ان هذا الوزيز هو معمر العرياني والموكد ...
12    الاتحادالافريقي لكرة القدم caf fifa دى جماهير ...
13    mounirhafi في واحد كان معبى الكراسي وزراء وطلع...
14    لاغاب نور القمر في عتمة اليل ينشاف نور البدر ف...
15    عطيتك قلب م عطيته لاحد غيرك غلطه و في ذمتي م ك...
16    الغريب في الامر انك مازلت تنتظر رغم كل هذه الخ...
17    almobark عقبال القمة دنيا باطما تاجر فبات 

In [34]:

train_data1 = pd.read_csv(Train_Data_Extended_File, sep=",")
# Cleaning Training Data 
train_data1[Tweets_Text_Col_Train] = train_data1[Tweets_Text_Col_Train].apply(lambda x:   data_cleaning(x))

# Removing un-needed feilds
if Tweets_Ids_Col_Train in train_data1.columns:
  del train_data1[Tweets_Ids_Col_Train]
train_data1.columns = [Tweets_Sentiment_Col_Train,Tweets_Text_Col_Train]

train_data1[Tweets_Text_Col_Train].head(50)


<ipython-input-29-f0cb5da77b0e>:19: DeprecationWarning: 'emoji.get_emoji_regexp()' is deprecated and will be removed in version 2.0.0. If you want to remove emoji from a string, consider the method emoji.replace_emoji(str, replace='').
To hide this warning, pin/downgrade the package to 'emoji~=1.6.3'
  em_split_emoji = emoji.get_emoji_regexp().split(em)


0     الزعل بيغير ملامحك بيغير نظرة العين بيغير شكلك...
1     halgawi DmfMohe ليس حبا في ايران بقدر ماهو نكا...
2     adalfahaduwail ابي اعرف الحاكم العربي المسلم ا...
3     sarmadbouchamou DimaSadek في الخطاب تبع سليم س...
4     FofaMahmoud مفيش الكلام ده في الزمن FofaMahmou...
5     elm عندي مشكله لما تبي اجد اقامة عامل حاولت ات...
6     الرياسة الفلسطينية تدعو السفراء العرب والمسلمي...
7     Hamed Alali التجنيس يضر اهل السنه في سوريا تغي...
8        سوف تبحث عني في شخص اخر سوف تبحث عن في شخص اخر
9     Bethoven كل ما املكه في تلك الحياة هو كرامتي و...
10    ان كان لك نصيب في شيء سيقلب اله كل الموازين لك...
11    الموكد ان هذا الوزيز هو معمر العرياني والموكد ...
12    الاتحادالافريقي لكرة القدم caf fifa دى جماهير ...
13    mounirhafi في واحد كان معبى الكراسي وزراء وطلع...
14    لاغاب نور القمر في عتمة اليل ينشاف نور البدر ف...
15    عطيتك قلب م عطيته لاحد غيرك غلطه و في ذمتي م ك...
16    الغريب في الامر انك مازلت تنتظر رغم كل هذه الخ...
17    almobark عقبال القمة دنيا باطما تاجر فبات 

In [35]:
import re
resultt = []
for row in train_data[Tweets_Text_Col_Train]:

  output = re.sub(emoji.get_emoji_regexp(), r"", row)
  output = re.sub(':[^>]+:', '', output)
  output = re.sub('<[^>]+>', '', output)
  resultt.append(output)

train_data['new_col'] = np.array(resultt)
train_data['new_col'].head(50)

<ipython-input-35-61a0ab371929>:5: DeprecationWarning: 'emoji.get_emoji_regexp()' is deprecated and will be removed in version 2.0.0. If you want to remove emoji from a string, consider the method emoji.replace_emoji(str, replace='').
To hide this warning, pin/downgrade the package to 'emoji~=1.6.3'
  output = re.sub(emoji.get_emoji_regexp(), r"", row)


0     الزعل بيغير ملامحك بيغير نظرة العين بيغير شكلك...
1     halgawi DmfMohe ليس حبا في ايران بقدر ماهو نكا...
2     adalfahaduwail ابي اعرف الحاكم العربي المسلم ا...
3     sarmadbouchamou DimaSadek في الخطاب تبع سليم س...
4     FofaMahmoud مفيش الكلام ده في الزمن FofaMahmou...
5     elm عندي مشكله لما تبي اجد اقامة عامل حاولت ات...
6     الرياسة الفلسطينية تدعو السفراء العرب والمسلمي...
7     Hamed Alali التجنيس يضر اهل السنه في سوريا تغي...
8        سوف تبحث عني في شخص اخر سوف تبحث عن في شخص اخر
9     Bethoven كل ما املكه في تلك الحياة هو كرامتي و...
10    ان كان لك نصيب في شيء سيقلب اله كل الموازين لك...
11    الموكد ان هذا الوزيز هو معمر العرياني والموكد ...
12    الاتحادالافريقي لكرة القدم caf fifa دى جماهير ...
13    mounirhafi في واحد كان معبى الكراسي وزراء وطلع...
14    لاغاب نور القمر في عتمة اليل ينشاف نور البدر ف...
15    عطيتك قلب م عطيته لاحد غيرك غلطه و في ذمتي م ك...
16    الغريب في الامر انك مازلت تنتظر رغم كل هذه الخ...
17    almobark عقبال القمة دنيا باطما تاجر فبات 

In [36]:
def clean_some_chars(text):
    search = ["أ","إ","آ","ة","_","-","/",".","،"," و "," يا ",'"',"ـ","'","ى","\\",'\n', '\t','"','?','؟','!'
              ,"[","]","{","}","*",":","#","$","€","£","~","<",">","/","|","'",",",'=','(',')','+','•',';','&','–','♦','%'
             ,'»','»','·',"\'ٓ'","'ٓ'","'","'ٓ"," 'ٓ'","ﻹ","ﻷ"]
    
    replace = ["ا","ا","ا","ه"," "," ","","",""," و"," يا","","","","ي","",' ', ' ',' ',' ',' ',' '
               ," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," ",' ',' ',' ',' ',' ',' ',' ',' ',' ',' '
              ,' ',' ',' ',' ', ' ',' ',' ',' ','لا','لا']
    
    text = text.replace('وو', 'و')
    text = text.replace('يي', 'ي')
    text = text.replace('اا', 'ا')
    text = text.replace('\'','')
    # removing numbers
    text = ''.join([i for i in text if not i.isdigit()])
    
    for i in range(0, len(search)):
        text = text.replace(search[i], replace[i])
    
    #trim    
    text = text.strip()

    return text

def clean_english_chars(text):
    search = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z'
             ,'A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Z','Y','Z']
    for i in range(0, len(search)):
        text = text.replace(search[i],' ')
    return text
    
def remove_unnecessary_spaces(text):
    return re.sub(' +',' ',text)

In [37]:
def remove_non_arabic_letters(text):
    
    '''
    ALEF_MADDA       = u'\u0622' 
    ALEF_HAMZA_ABOVE = u'\u0623' 
    WAW_HAMZA        = u'\u0624' 
    ALEF_HAMZA_BELOW = u'\u0625' 
    YEH_HAMZA        = u'\u0626' 
    ALEF             = u'\u0627' 
    BEH              = u'\u0628' 
    TEH_MARBUTA      = u'\u0629' 
    TEH              = u'\u062a' 
    THEH             = u'\u062b' 
    JEEM             = u'\u062c' 
    HAH              = u'\u062d' 
    KHAH             = u'\u062e' 
    DAL              = u'\u062f' 
    THAL             = u'\u0630' 
    REH              = u'\u0631' 
    ZAIN             = u'\u0632' 
    SEEN             = u'\u0633' 
    SHEEN            = u'\u0634' 
    SAD              = u'\u0635' 
    DAD              = u'\u0636' 
    TAH              = u'\u0637' 
    ZAH              = u'\u0638' 
    AIN              = u'\u0639' 
    GHAIN            = u'\u063a' 
    TATWEEL          = u'\u0640' 
    FEH              = u'\u0641' 
    QAF              = u'\u0642' 
    KAF              = u'\u0643' 
    LAM              = u'\u0644' 
    MEEM             = u'\u0645' 
    NOON             = u'\u0646' 
    HEH              = u'\u0647' 
    WAW              = u'\u0648' 
    ALEF_MAKSURA     = u'\u0649' 
    YEH              = u'\u064a' 
    MADDA_ABOVE      = u'\u0653' 
    HAMZA_ABOVE      = u'\u0654' 
    HAMZA_BELOW      = u'\u0655' 
    LAM_ALEF                     = u'\ufefb' 
    LAM_ALEF_HAMZA_ABOVE         = u'\ufef7' 
    LAM_ALEF_HAMZA_BELOW         = u'\ufef9' 
    LAM_ALEF_MADDA_ABOVE         = u'\ufef5' 
    '''
      
    regex = re.compile(r'[\u0622\u0623\u0624\u0625\u0626\u0627\u0628\u0629\u062a\u062b\u062c\u062d\u062e\u062f\u0630\u0631\u0632\u0633\u0634\u0635\u0636\u0637\u0638\u0639\u063a\u0640\u0641\u0642\u0643\u0644\u0645\u0646\u0647\u0648\u0649\u064a\u0653\u0654\u0655\ufefb\ufef7\ufef9\ufef5]')
    # removing Arabic letters from the text and storing the result in the varialbe: unwanted_str .
    unwanted_str = regex.sub(' ',text)
    # Creating a list containing all of the unwanted characters, letters and symbols.
    unwanted_list_of_strs = list(unwanted_str.replace(" ", ""))
    # Cleaning the unwanted list of characters out of the text
    for i in range(0, len(unwanted_list_of_strs)):
        text = text.replace(unwanted_list_of_strs[i], " ")
    
    text = remove_unnecessary_spaces(text)
    
    return text

In [38]:
def concatenate_list_into_string(lis_strs):
    result = ""
    for el in lis_strs:
        result += " " + el
    return result
def remove_single_letters(text):
    words = text.split(' ')
    waw = 'و'
    for word in words:
        if len(word.strip()) == 1:
            if word != waw:
                words.remove(word)
    text = concatenate_list_into_string(words)
    return text

In [39]:
final = []
for row in train_data['new_col']:
  def clean_text(row):
    # removing some unuseful chars
    text = clean_some_chars(row)
    # removing english chars
    text = clean_english_chars(text)
    # removing tashkeel
    text = araby.strip_tashkeel(text)
    # removing longation
    text = araby.strip_tatweel(text)
    # removing unwanted spaces
    text = remove_unnecessary_spaces(text)
    # removing non-arabic characters
    text = remove_non_arabic_letters(text)
    # removing single unwanted letters
    text = remove_single_letters(text)
    # returning result
    return text  

 
    #text = text
    #final.append(text)

#train_data['new_col'] = np.array(final)
#train_data['new_col'].head(50)

In [40]:
train_data['new_col'].dropna()

0        الزعل بيغير ملامحك بيغير نظرة العين بيغير شكلك...
1        halgawi DmfMohe ليس حبا في ايران بقدر ماهو نكا...
2        adalfahaduwail ابي اعرف الحاكم العربي المسلم ا...
3        sarmadbouchamou DimaSadek في الخطاب تبع سليم س...
4        FofaMahmoud مفيش الكلام ده في الزمن FofaMahmou...
                               ...                        
73609    RaeidKh كلم هذا الحساب myeleservices يفيدك عشا...
73610    احضني خايف اطيح من ثقل ايامي علي احض خايف اطيح...
73611    نجاح الموسم من نجاح الترفيه و قوة تنظيمه و انش...
73612    دعواتكم لفقيده نوره المالكيكاتب السيناريو علاء...
73613    SaudiMOH هل ينصح بتناول البروفين لمن يعاني بال...
Name: new_col, Length: 73614, dtype: object

In [41]:
train_data['new_col'] = train_data['new_col'].astype(str)

In [42]:
train_data['new_col']

0        الزعل بيغير ملامحك بيغير نظرة العين بيغير شكلك...
1        halgawi DmfMohe ليس حبا في ايران بقدر ماهو نكا...
2        adalfahaduwail ابي اعرف الحاكم العربي المسلم ا...
3        sarmadbouchamou DimaSadek في الخطاب تبع سليم س...
4        FofaMahmoud مفيش الكلام ده في الزمن FofaMahmou...
                               ...                        
73609    RaeidKh كلم هذا الحساب myeleservices يفيدك عشا...
73610    احضني خايف اطيح من ثقل ايامي علي احض خايف اطيح...
73611    نجاح الموسم من نجاح الترفيه و قوة تنظيمه و انش...
73612    دعواتكم لفقيده نوره المالكيكاتب السيناريو علاء...
73613    SaudiMOH هل ينصح بتناول البروفين لمن يعاني بال...
Name: new_col, Length: 73614, dtype: object

In [43]:
import pyarabic.araby as araby

In [44]:
train_data['new_col'] = train_data['new_col'].apply(clean_text)

In [45]:
train_data['new_col'].head(50)

0      الزعل بيغير ملامحك بيغير نظره العين بيغير شكل...
1       ليس حبا في ايران بقدر ماهو نكايه بترامب وحزب...
2       ابي اعرف الحاكم العربي المسلم اشلون ينام ماي...
3       في الخطاب تبع سليم سعاده حطت عالتويتر شو قال...
4           مفيش الكلام ده في الزمن مفيش كلام ده في زمن
5       عندي مشكله لما تبي اجد اقامه عامل حاولت اتوا...
6      الرياسه الفلسطينيه تدعو السفرا العرب والمسلمي...
7       التجنيس يضر اهل السنه في سوريا تغير التركيبه...
8        سوف تبحث عني في شخص اخر سوف تبحث عن في شخص اخر
9       كل ما املكه في تلك الحياه هو كرامتي وكلمتي و...
10     ان كان لك نصيب في شي سيقلب اله كل الموازين لك...
11     الموكد ان هذا الوزيز هو معمر العرياني والموكد...
12     الاتحادالافريقي لكره القدم دي جماهير الهلال ا...
13      في واحد كان معبي الكراسي وزرا وطلع افشل واحد...
14     لاغاب نور القمر في عتمه اليل ينشاف نور البدر ...
15     عطيتك قلب عطيته لاحد غيرك غلطه وفي ذمتي كنت ق...
16     الغريب في الامر انك مازلت تنتظر رغم كل هذه ال...
17      عقبال القمه دنيا باطما تاجر فبات المغرب 

In [46]:
import re
result = []
for row in train_data['new_col']:
  output= re.sub(r'\s*[A-Za-z]+\b', '' , row)
  output = ''.join ([t for t in output if t not in ['ٓ']])
  output= re.sub(r'⠀', '' , output)  
  output= re.sub(r'ﻻ', '' , output) 
  output= re.sub(r'\u200c', '' , output)
  output= re.sub(r'ﷺ', '' , output)
  output= re.sub(r'•', '' , output) 
  output= re.sub(r'گ', 'ك' , output)
  output= re.sub(r'—', '' , output) 
  output= re.sub(r'️', '' , output)
  output= re.sub(r'﴾', '' , output) 
  output= re.sub(r'ٰ', '' , output) 
  output= re.sub(r'♡', '' , output) 
  output= re.sub(r'❀', '' , output) 
  output= re.sub(r'ᅠ', '' , output)  
  output= re.sub(r'ﺀ', 'ء' , output) 
  output= re.sub(r'ﻣ', 'م' , output) 
  output= re.sub(r'ﻭ', 'و' , output)
  output= re.sub(r'ﺿ', 'ض' , output) 
  output= re.sub(r'ﻴ', 'ي' , output) 
  output= re.sub(r'ﻖ', 'ق' , output) 
  output= re.sub(r'☜', '' , output) 
  output= re.sub(r'ﺛ', 'ث' , output) 
  output= re.sub(r'ﻧ', 'ن' , output)
  output= re.sub(r'’', '' , output) 
  output= re.sub(r'…', '' , output)
  output= re.sub(r'ﻳ', 'ي' , output) 
  output= re.sub(r'ﺸ', 'ش' , output) 
  output= re.sub(r'ﺮ', 'ر' , output)
  output= re.sub(r'ﻕ', 'ق' , output)  
  output= re.sub(r'ﺗ', 'ت' , output) 
  output= re.sub(r'ﻔ', 'ف' , output)  
  output= re.sub(r'ﺎ', '' , output) 
  output= re.sub(r'ﺅ', 'ؤ' , output) 
  output= re.sub(r'ﻌ', 'ع' , output) 
  output= re.sub(r'ﺰ', 'ز' , output)  
  output= re.sub(r'ﻑ', 'ف' , output) 
  output= re.sub(r'ﺃ', 'أ' , output) 
  output= re.sub(r'ﻼ', 'لا' , output) 
  output= re.sub(r'˝', '"' , output) 
  output= re.sub(r'چ', 'ج' , output) 
  output= re.sub(r'ہ', 'ه' , output) 
  output= re.sub(r's', '' , output) 
  output= re.sub(r'ı', '' , output) 
  output= re.sub(r'r', '' , output) 
  output= re.sub(r'ㅤ', ' ' , output) 
  output= re.sub(r'ﺍ', 'ا' , output) 
  output= re.sub(r'ﻟ', 'ل' , output) 
  output= re.sub(r'ﻀ', 'ض' , output) 
  output= re.sub(r'ﻤ', ' م' , output) 
  output= re.sub(r'ﺻ', 'ص' , output) 
  output= re.sub(r'ﺒ', 'ب' , output)  
  output= re.sub(r'ﺢ', 'ح' , output) 
  output= re.sub(r'ﻓ', 'ف' , output) 
  output= re.sub(r'ﻲ', 'ي' , output) 
  output= re.sub(r'ﺩ', 'د' , output) 
  output= re.sub(r'ﺭ', 'ر' , output) 
  output= re.sub(r'ﺳ', 'س' , output) 
  output= re.sub(r'ﻨ', 'ن' , output) 
  output= re.sub(r'ﺤ', 'ح' , output) 
  output= re.sub(r'ﻮ', 'و' , output) 
  output= re.sub(r'ﻘ', 'ق' , output)  
  output= re.sub(r'ﻄ', 'ط' , output) 
  output= re.sub(r'ی', 'ى' , output) 
  output= re.sub(r'ڼ', 'ن' , output) 
  output= re.sub(r'ژ', 'ز' , output) 
  output= re.sub(r'٬', ',' , output) 
  output= re.sub(r'٪', '' , output) 
  output= re.sub(r'﴿', '' , output) 
  output= re.sub(r'\u2066', '' , output) 
  output= re.sub(r'\u2069', '' , output) 
  output= re.sub(r'ڪ', 'ك' , output) 
  output= re.sub(r'٫', ',' , output)  
  output= re.sub(r'ھ', 'ه' , output) 
  output= re.sub(r'ﮐ', 'ك' , output) 
  output= re.sub(r'ﺣ', 'ح' , output) 
  output= re.sub(r'ﮬ', 'ه' , output) 
  output= re.sub(r'ﻗ', 'ق' , output) 
  output= re.sub(r'ﻋ', 'ع' , output) 
  output= re.sub(r'ﺑ', 'ب' , output) 
  output= re.sub(r'ﭑ', 'أ' , output)  
  output= re.sub(r'H', '' , output) 
  output= re.sub(r'o', '' , output) 
  output= re.sub(r'a', '' , output) 
  output= re.sub(r'c', '' , output) 
  output= re.sub(r't', '' , output) 
  output= re.sub(r'i', '' , output)
  output= re.sub(r'n', '' , output)
  output= re.sub(r"'ٓ", '' , output)
  output= re.sub(r"ﻹ", 'لا' , output)
  output= re.sub(r"لأ", 'لا' , output)


  output = output.rstrip()

  def clean_text(output):
    # removing some unuseful chars
    output = clean_some_chars(output)
    # removing english chars
    output = clean_english_chars(output)
    # removing tashkeel
    output = araby.strip_tashkeel(output)
    # removing longation
    output = araby.strip_tatweel(output)
    # removing unwanted spaces
    output = remove_unnecessary_spaces(output)
    # removing non-arabic characters
    output = remove_non_arabic_letters(output)
    # removing single unwanted letters
    output = remove_single_letters(output)
    # returning result
    return output  





  
  result.append(output)

train_data['new_col'] = np.array(result)
train_data['new_col'].head(10)

0     الزعل بيغير ملامحك بيغير نظره العين بيغير شكل...
1      ليس حبا في ايران بقدر ماهو نكايه بترامب وحزب...
2      ابي اعرف الحاكم العربي المسلم اشلون ينام ماي...
3      في الخطاب تبع سليم سعاده حطت عالتويتر شو قال...
4          مفيش الكلام ده في الزمن مفيش كلام ده في زمن
5      عندي مشكله لما تبي اجد اقامه عامل حاولت اتوا...
6     الرياسه الفلسطينيه تدعو السفرا العرب والمسلمي...
7      التجنيس يضر اهل السنه في سوريا تغير التركيبه...
8       سوف تبحث عني في شخص اخر سوف تبحث عن في شخص اخر
9      كل ما املكه في تلك الحياه هو كرامتي وكلمتي و...
Name: new_col, dtype: object

In [ ]:
lines = train_data['new_col']

results = list()
for idx, line in enumerate(lines):
    line = remove_diacritics(line.strip())

    x = predict(line, model)
    results.append(x)

train_data['new_col'] = np.array(result)
train_data['new_col'].head(50)

Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 111s 111s/step


In [ ]:
print (train_data['new_col'].iloc[[42]] )

In [ ]:
CHARACTERS_MAPPING

In [ ]:
CHARACTERS_MAPPING['ء']

In [ ]:
print(predict('ء', model))

In [ ]:
diac = []
for row in train_data['new_col']:
    row = remove_diacritics(row.strip())
    x = predict(row, model)
    diac.append(x)
lines = train_data['new_col']
results = list()
for idx, line in enumerate(lines):
    line = remove_diacritics(line.strip())
    x = predict(line, model)
    results.append(x)

train_data1['new_col'] = np.array(diac)
train_data1['new_col'].head(50)

In [ ]:
# First setting the max_len , will be useful later for BERT Model
Extra_Len = 6 # an extra padding in length , found to be useful for increasing F-score
Max_Len = train_data['new_col'].str.split().str.len().max() + Extra_Len
print(Max_Len)

#Spliting the Training data
Test_Size = 0

Test_Size = 0.001  # low percentage to keep the training data as large as possible,
                     # the value 0.001 found to be best for F-Score with extended data

Rand_Seed = 42 

train_set, evaluation_set = train_test_split( train_data, test_size= Test_Size, random_state= Rand_Seed)
print("Train set: ")
print(train_set[Tweets_Sentiment_Col_Train].value_counts())
print("---------------------------")
print ("Evaluation set: ")
print (evaluation_set[Tweets_Sentiment_Col_Train].value_counts())

In [ ]:
# preparing test_data

Tweets_Ids_Col_Test = "Tweet_id"
Tweets_Text_Col_Test = "Text"
Test_Data_File = "test1_with_text.csv"

test_data = pd.read_csv(Test_Data_File, sep=",")
test_data.columns = [Tweets_Ids_Col_Test,Tweets_Text_Col_Test]

test_data[Tweets_Text_Col_Test] = test_data[Tweets_Text_Col_Test].apply(lambda x:   data_cleaning(x))
test_data[Tweets_Text_Col_Test].head(50)

In [ ]:
Model_Used = "UBC-NLP/MARBERT"
Task_Name = "classification"

class Dataset:
    def __init__(
        self,
        name,
        train,
        test,
        label_list,
    ):
        self.name = name
        self.train = train
        self.test = test
        self.label_list = label_list
class BERTModelDataset(Dataset):
    def __init__(self, text, target, model_name, max_len, label_map):
      super(BERTModelDataset).__init__()
      self.text = text
      self.target = target
      self.tokenizer_name = model_name
      self.tokenizer = AutoTokenizer.from_pretrained(model_name)
      self.max_len = max_len
      self.label_map = label_map
  
    def __len__(self):
      return len(self.text)

    def __getitem__(self,item):
      text = str(self.text[item])
      text = " ".join(text.split())
    
      encoded_review = self.tokenizer.encode_plus(
      text,
      max_length= self.max_len,
      add_special_tokens= True,
      return_token_type_ids=False,
      pad_to_max_length=True,
      truncation='longest_first',
      return_attention_mask=True,
      return_tensors='pt' )
      input_ids = encoded_review['input_ids'].to(device)
      attention_mask = encoded_review['attention_mask'].to(device)

      return InputFeatures(input_ids=input_ids.flatten(), attention_mask=attention_mask.flatten(), label=self.label_map[self.target[item]])

In [ ]:
def model_init():
  return AutoModelForSequenceClassification.from_pretrained(Model_Used, return_dict=True, num_labels=len(label_map))

def compute_metrics(p): #p should be of type EvalPrediction
  preds = np.argmax(p.predictions, axis=1)
  assert len(preds) == len(p.label_ids)
  print(classification_report(p.label_ids,preds))
  #print(confusion_matrix(p.label_ids,preds))

  macro_f1_pos_neg = f1_score(p.label_ids,preds,average='macro',labels=[1,2])
  macro_f1 = f1_score(p.label_ids,preds,average='macro')
  macro_precision = precision_score(p.label_ids,preds,average='macro')
  macro_recall = recall_score(p.label_ids,preds,average='macro')
  acc = accuracy_score(p.label_ids,preds)
  return {
      'macro_f1' : macro_f1,
      'macro_f1_pos_neg' : macro_f1_pos_neg,  
      'macro_precision': macro_precision,
      'macro_recall': macro_recall,
      'accuracy': acc
  }

def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

In [ ]:
label_list = list(train_set[Tweets_Sentiment_Col_Train].unique())

print(label_list)
print(train_set[Tweets_Sentiment_Col_Train].value_counts())

data_set = Dataset( "KAUST", train_set, evaluation_set, label_list )

label_map = { v:index for index, v in enumerate(label_list) }
print(label_map)

train_dataset = BERTModelDataset(train_set[Tweets_Text_Col_Train].to_list(),
                                 train_set[Tweets_Sentiment_Col_Train].to_list(),Model_Used,Max_Len,label_map)

evaluation_dataset = BERTModelDataset(evaluation_set[Tweets_Text_Col_Train].to_list(),
                                      evaluation_set[Tweets_Sentiment_Col_Train].to_list(),Model_Used,Max_Len,label_map)

In [ ]:
#define training arguments
training_args = TrainingArguments("./train")
training_args.lr_scheduler_type = 'cosine'
training_args.evaluate_during_training = True
training_args.adam_epsilon =1e-8
training_args.learning_rate = 1.215e-05 # use this with extended data
training_args.fp16 = True
training_args.per_device_train_batch_size = 16 #64 
training_args.per_device_eval_batch_size = 16 # 64 
training_args.gradient_accumulation_steps = 2
training_args.num_train_epochs= 2
training_args.warmup_steps = 0 
training_args.evaluation_strategy = EvaluationStrategy.EPOCH
training_args.logging_steps = 200
training_args.save_steps = 100000 
training_args.seed = 42 
training_args.disable_tqdm = False

In [ ]:
training_args.dataloader_pin_memory = False
gc.collect()
torch.cuda.empty_cache()
set_seed(Rand_Seed) 

trainer = Trainer(
    model = model_init(),
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset= evaluation_dataset,
    compute_metrics=compute_metrics
)

print(training_args.seed)

In [ ]:
print(Max_Len)
print(training_args.learning_rate)
print(training_args.adam_epsilon)
print(training_args.warmup_steps)
#wandbkey if needed (depend on the transformers package version) = 0a58b374c46a154de1ba77c8634c6be279a9dcdb
trainer.train()

In [ ]:
# first define the predection method
def predicts(text, tokenizer):
 
  encoded_review = tokenizer.encode_plus(
    text,
    max_length=Max_Len,
    add_special_tokens=True,
    return_token_type_ids=False,
    pad_to_max_length=True, #True,
    truncation='longest_first',
    return_attention_mask=True,
    return_tensors='pt'
  )

  input_ids = encoded_review['input_ids'].to(device) #(input_ids + ([tokenizer.pad_token_id] * padding_length)).to(device)  
  attention_mask = encoded_review['attention_mask'].to(device)
    

  output = trainer.model(input_ids, attention_mask)
  _, prediction = torch.max(output[0], dim=1)
  return prediction[0]


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(Model_Used)
tweet = "من اروع و اجمل الفعاليات لي حضرتها حقت موسم جد"
pre = predicts(tweet,tokenizer)
pre_txt = label_list[pre]
print(pre_txt)

In [ ]:
print(predict(tweet, model))